# CSP Solver for Wordle game

### Useful links :
- [Medium article](https://medium.com/better-programming/beating-wordle-constraint-programming-ef0b0b6897fe#:~:text=Beating%20Wordle%3A%20Constraint%20Programming,Wordle%20solver%20do%20its%20thing)
- [Sample dataset (GitHub)](https://github.com/dwyl/english-words)


In [8]:
# =========================
# Chargement et préparation du dictionnaire de mots
# =========================

import pandas as pd
from collections import Counter, defaultdict
import itertools

# Charger le fichier contenant tous les mots anglais
df_words = pd.read_fwf("words_alpha.txt", names=["word"])

# Ne garder que les mots de 5 lettres
df_words_5 = df_words[df_words["word"].str.len() == 5]

# Convertir chaque mot en tuple d'entiers (a=0, b=1, ..., z=25)
words_tuples = [tuple(ord(c) - ord('a') for c in w) for w in df_words_5["word"]]

# -------------------------
# Statistiques des lettres
# -------------------------
# Fréquence des lettres par position
freq_by_position = [defaultdict(int) for _ in range(5)]
for w in words_tuples:
    for i, c in enumerate(w):
        freq_by_position[i][c] += 1

# Fréquence globale de chaque lettre
freq_letters = Counter(itertools.chain.from_iterable(words_tuples))

# Normaliser les fréquences globales
n_words = len(words_tuples)
freq_letters = {c: count / n_words for c, count in freq_letters.items()}

# Normaliser les fréquences par position
for i in range(5):
    total_pos = sum(freq_by_position[i].values())
    freq_by_position[i] = {c: count / total_pos for c, count in freq_by_position[i].items()}


In [9]:
# =========================
# CSP Solver pour Wordle
# =========================

import random
from collections import defaultdict, Counter
from ortools.sat.python import cp_model

# -------------------------
# Fonction de feedback
# -------------------------
def get_feedback(guess, target):
    """
    Retourne le feedback d'une tentative par rapport au mot cible.
    'G' = verte (correcte position)
    'Y' = jaune (présente mais mauvaise position)
    'B' = noire (absente)
    """
    feedback = []
    target_counts = Counter(target)

    # Lettres vertes
    for g_char, t_char in zip(guess, target):
        if g_char == t_char:
            feedback.append('G')
            target_counts[g_char] -= 1
        else:
            feedback.append('B')

    # Lettres jaunes
    for i, g_char in enumerate(guess):
        if feedback[i] == 'B' and target_counts[g_char] > 0:
            feedback[i] = 'Y'
            target_counts[g_char] -= 1

    return feedback


# -------------------------
# Mise à jour du modèle CSP
# -------------------------
def update_model(model, position_vars, guess, feedback):
    """
    Applique les contraintes au modèle CSP selon le feedback
    """
    letter_counts = defaultdict(int)
    gray_positions = defaultdict(list)

    for i, (char, fb) in enumerate(zip(guess, feedback)):
        if fb == 'G':
            model.Add(position_vars[i] == char)
            letter_counts[char] += 1
        elif fb == 'Y':
            model.Add(position_vars[i] != char)
            letter_counts[char] += 1
        elif fb == 'B':
            gray_positions[char].append(i)

    # Contraintes pour lettres noires
    for char, positions in gray_positions.items():
        for i in positions:
            model.Add(position_vars[i] != char)

    # Nombre minimum d'occurrences pour lettres vertes/jaunes
    for char, count in letter_counts.items():
        occurs = [model.NewBoolVar(f'occurs_{i}_{char}') for i in range(5)]
        for i in range(5):
            if i not in gray_positions.get(char, []):
                model.Add(position_vars[i] == char).OnlyEnforceIf(occurs[i])
                model.Add(position_vars[i] != char).OnlyEnforceIf(occurs[i].Not())
        model.Add(sum(occurs) >= count)


# -------------------------
# Fonction heuristique
# -------------------------
def update_heuristic(model, position_vars):
    """
    Fonction objectif pour maximiser la qualité de la prochaine tentative
    """
    coef_pos_freq = 1000
    coef_letter_freq = 2000
    coef_duplicates = 500

    objective_terms = []

    # Score par position et lettre
    for i, var in enumerate(position_vars):
        for c in range(26):
            is_char = model.NewBoolVar(f'pos_{i}_char_{c}')
            model.Add(var == c).OnlyEnforceIf(is_char)
            model.Add(var != c).OnlyEnforceIf(is_char.Not())
            score = int(coef_pos_freq * freq_by_position[i][c] + coef_letter_freq * freq_letters[c])
            objective_terms.append(is_char * score)

    # Gestion des doublons
    num_duplicates = model.NewIntVar(0, 25, 'num_duplicates')
    duplicates = []
    for c in range(26):
        count = sum([model.NewBoolVar(f'pos_{i}_char_{c}') for i in range(5)])
        is_dup = model.NewBoolVar(f'is_dup_{c}')
        model.Add(count > 1).OnlyEnforceIf(is_dup)
        model.Add(count <= 1).OnlyEnforceIf(is_dup.Not())
        duplicates.append(is_dup)

    model.Add(num_duplicates == sum(duplicates))
    objective_terms.append(-coef_duplicates * num_duplicates)

    model.Maximize(sum(objective_terms))


# -------------------------
# Filtrage des mots valides
# -------------------------
def filter_valid_words(words_list, guess, feedback):
    """
    Retourne la liste des mots valides en fonction du feedback
    """
    valid_words = []
    for word in words_list:
        valid = True

        # Vertes
        for i in range(5):
            if feedback[i] == 'G' and word[i] != guess[i]:
                valid = False
                break
        if not valid:
            continue

        # Jaunes
        for i in range(5):
            if feedback[i] == 'Y' and (word[i] == guess[i] or guess[i] not in word):
                valid = False
                break
        if not valid:
            continue

        # Noires
        gray_chars = [guess[i] for i in range(5) if feedback[i] == 'B']
        for char in gray_chars:
            if char in word and char not in [guess[i] for i in range(5) if feedback[i] in ('G', 'Y')]:
                valid = False
                break

        if valid:
            valid_words.append(word)
    return valid_words


# -------------------------
# Fonction principale
# -------------------------
def solve_wordle(target_word, max_attempts=6, print_output=True):
    target_int = [ord(c) - ord('a') for c in target_word]
    valid_words = words_tuples.copy()

    model = cp_model.CpModel()
    position_vars = [model.NewIntVar(0, 25, f'pos_{i}') for i in range(5)]

    status_dict = {
        cp_model.OPTIMAL: "OPTIMAL",
        cp_model.FEASIBLE: "FEASIBLE",
        cp_model.INFEASIBLE: "INFEASIBLE",
        cp_model.MODEL_INVALID: "MODEL_INVALID",
        cp_model.UNKNOWN: "UNKNOWN"
    }

    if print_output:
        print(f"\nFreq pos : {freq_by_position}")
        print(f"Freq lettres : {freq_letters}\n")

    for attempt in range(max_attempts):
        if print_output:
            print(f"Mots possibles : {len(valid_words)}")
            print(f"Le mot cible est dans le dataset ? {tuple(target_int) in valid_words}")

        model.AddAllowedAssignments(position_vars, valid_words)
        update_heuristic(model, position_vars)

        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        if print_output:
            print(f"Status = {status_dict.get(status,'UNKNOWN')}")

        if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            guess = tuple(solver.Value(var) for var in position_vars)
            valid_words.remove(guess)
            guess_str = ''.join([chr(c + ord('a')) for c in guess])
            feedback = get_feedback(guess, target_int)
            if print_output:
                print(f"Tentative {attempt+1}: {guess_str} → {feedback}")

            if feedback == ['G'] * 5:
                print(f"Mot trouvé en {attempt+1} tentatives !")
                return attempt + 1

            valid_words = filter_valid_words(valid_words, guess, feedback)
            update_model(model, position_vars, guess, feedback)
        else:
            print("Modèle impossible à résoudre. Fin.")
            return attempt + 1

    print(f"Échec pour {target_word} après {max_attempts} tentatives.")
    return max_attempts


# -------------------------
# Test aléatoire
# -------------------------
random_word = random.choice(df_5letters["word"].to_list())
print(f"Mot aléatoire : {random_word}")
solve_wordle(random_word, max_attempts=5)


Mot aléatoire : frosk

Freq pos : [{0: 0.07373908674078261, 1: 0.0716663526160417, 2: 0.07518371961560204, 3: 0.05031091011871114, 4: 0.026443062621694616, 5: 0.04296212549462974, 6: 0.04629106211921362, 7: 0.03586458137051693, 24: 0.010489290873688838, 8: 0.01890584762263677, 9: 0.016330632497958672, 10: 0.029709189121286353, 11: 0.04264807486966899, 12: 0.053325796118334275, 13: 0.025500910746812388, 14: 0.020978581747377677, 15: 0.0592927579925884, 16: 0.005338860624332643, 17: 0.04277369511965329, 18: 0.11387475661076565, 19: 0.06161673261729791, 20: 0.020601720997424786, 21: 0.018026505872746686, 22: 0.029395138496325607, 23: 0.0016958733747880158, 25: 0.007034733999120658}, {0: 0.18032786885245902, 1: 0.006846303624144212, 2: 0.015953771748005777, 3: 0.008542176998932227, 4: 0.12379875635952516, 5: 0.0025124049996859492, 6: 0.006406632749199171, 7: 0.04522328999434709, 24: 0.017586834997801646, 8: 0.10483009861189624, 9: 0.0011933923748508259, 10: 0.006343822624207022, 11: 0.0543

5